In [ ]:
import glob
import cv2
import numpy as np
import os
import re
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir("/content/drive/My Drive") 
path = "/content/drive/My Drive"
os.chdir(path)
os.listdir(path)

['Colab Notebooks',
 'result.txt',
 'results.pickle',
 'fall_detection',
 'labels_urfd_tf.h5',
 'Fall 2019 ECE 391 Final Exam Review.gdoc',
 'data',
 'video_image',
 'VGG',
 'videos to images.ipynb',
 'csv']

In [ ]:
# cur_path = os.path.dirname(os.path.abspath("__file__"))
# par_path = os.path.abspath(os.path.join(cur_path,os.path.pardir))
# print (cur_path)
# print (par_path)
# data_kind = "Movie Dataset"
# data_path = par_path+"/"+ data_kind + " opflow"
# os.listdir(data_path)

data_kind = "Movie Dataset"
data_path = path+"/video_image"+ "/"+ data_kind + " opflow"
print (data_path)
os.listdir(data_path)


/content/drive/My Drive/video_image/Movie Dataset opflow


['fight', 'noFight']

In [ ]:
print ("fight data = ",len(glob.glob(r"%s"%(data_path+"/fight/*.mp4"))))
print ("noFight data = ",len(glob.glob(r"%s"%(data_path+"/noFight/*.mp4"))))

fight data =  100
noFight data =  93


In [ ]:
cur_path=os.getcwd()
file_kind = 'Movie Dataset opflow'
data_path = cur_path+'/video_image/'+file_kind
print (data_path)

/content/drive/My Drive/video_image/Movie Dataset opflow


In [ ]:
fight_files = glob.glob(data_path+'/fight'+'/*mp4')
noFight_files = glob.glob(data_path+'/noFight'+'/*mp4')
total_files = fight_files+noFight_files

In [ ]:
# size for potical flow batch
L = 10

In [ ]:
# key for the string sort, it will get the digit 
# from the string and then sort it 
def sort_key(s):
    if s:
        try:
            c = re.findall(r"\d+", s)[0]

        except:
            c = -1
        return int(c)

In [ ]:
# generator function for the images
def generator(list1, lits2):
    '''
    Auxiliar generator: returns the ith element of both given list with
         each call to next() 
    '''
    for x,y in zip(list1,lits2):
        yield x, y

In [ ]:
# convert the optical flow into the csv file
def convert_data_to_csv(file,file_kind):
    # count for the loop number
    count = 0
    for video_path in file:
        count+=1
        x_images = glob.glob(video_path + '/x_*.jpg')
        x_images.sort(key=sort_key)
        y_images = glob.glob(video_path + '/y_*.jpg')
        y_images.sort(key=sort_key)
        nb_stacks = len(x_images)-L+1
        flow = np.zeros(shape=(224,224,2*L,nb_stacks), dtype=np.float64)
        gen = generator(x_images,y_images)
        # print(video_path)
        # print(np.shape(x_images),np.shape(y_images))
        for i in range(len(x_images)):
            flow_x_file, flow_y_file = next(gen)
            img_x = cv2.imread(flow_x_file, cv2.IMREAD_GRAYSCALE)
            img_y = cv2.imread(flow_y_file, cv2.IMREAD_GRAYSCALE)
            # Assign an image i to the jth stack in the kth position, but also
            # in the j+1th stack in the k+1th position and so on
            # (for sliding window) 
            for s in list(reversed(range(min(L,i+1)))):
                if i-s < nb_stacks:
                    flow[:,:,2*s,i-s] = img_x
                    flow[:,:,2*s+1,i-s] = img_y
            del img_x,img_y
            gc.collect()                
        
        # Should notice that each flow represents all the optical flow data from 
        # one document (video)
        flow = np.transpose(flow, (3, 0, 1, 2))

        # To transfer it to csv form, we need to reshape it into 2D array
        flow = np.reshape(flow,(-1,224))
        
        # get the name of the video 
        file_name = re.search(r'ght/(.*?)(\.mp4)',video_path).group(1)
        
        if "noFight" in video_path:
            outPutDirName=path+'/csv/'+file_kind+'/noFight/'
            if not os.path.exists(outPutDirName):
                # If it doesn't exit, create it
                os.makedirs(outPutDirName)
            out_path = '%s.csv'%os.path.join(outPutDirName,file_name)
        else :
            outPutDirName=path+'/csv/'+file_kind+'/fight/'
            if not os.path.exists(outPutDirName):
                # If it doesn't exit, create it
                os.makedirs(outPutDirName)
            out_path = '%s.csv'%os.path.join(outPutDirName,file_name)
        print('%i/%i'%(count,len(file)))
        
        # load the 2d arrary into csv
        # fmt="%d" means data are stored as interger
        np.savetxt(out_path, flow, fmt="%d",delimiter = ',')


In [ ]:
convert_data_to_csv(total_files,file_kind)

1/193
2/193
3/193
4/193
5/193
6/193


KeyboardInterrupt: ignored